In [10]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np
import argparse
import matplotlib.pyplot as plt
import os
from imutils import paths

In [3]:
# Dataset directory and categories
DIRECTORY = r"C:\Users\HP\Desktop\projects\face_mask_detector\dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [4]:
# initialize the initial learning rate, number of epochs to train for,and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [5]:
data = []
labels = []

In [6]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        imgPath = os.path.join(path, img)
        image = load_img(imgPath, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [7]:
# One-Hot Encoding

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)


In [8]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [9]:
# constructing the training image generator for data augmentation

aug = ImageDataGenerator(
rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest")

In [23]:
#x_train

In [10]:
# MobileNetV2

baseModel = MobileNetV2(weights="imagenet", include_top=False, 
                        input_tensor=Input(shape=(224,224,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [11]:
# compile model

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train head of the network
H = model.fit(
aug.flow(x_train, y_train, batch_size=BS),
steps_per_epoch=len(x_train) // BS,
validation_data=(x_test, y_test),
validation_steps=len(x_test) // BS,
epochs=EPOCHS)

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
102/102 [==============================] - 334s 3s/step - loss: 0.3880 - accuracy: 0.8600 - val_loss: 0.1341 - val_accuracy: 0.9829
Epoch 2/20
102/102 [==============================] - 333s 3s/step - loss: 0.1451 - accuracy: 0.9658 - val_loss: 0.0751 - val_accuracy: 0.9853
Epoch 3/20
102/102 [==============================] - 328s 3s/step - loss: 0.1015 - accuracy: 0.9723 - val_loss: 0.0579 - val_accuracy: 0.9890
Epoch 4/20
102/102 [==============================] - 344s 3s/step - loss: 0.0819 - accuracy: 0.9763 - val_loss: 0.0523 - val_accuracy: 0.9878
Epoch 5/20
102/102 [==============================] - 350s 3s/step - loss: 0.0688 - accuracy: 0.9809 - val_loss: 0.0465 - val_accuracy: 0.9902
Epoch 6/20
102/102 [==============================] - 371s 4s/step - loss: 0.0567 - accuracy: 0.9846 - val_loss: 0.0445 - val_accuracy: 0.9878
Epoch 7/20
102/102 [==============================] - 437s 4s/step - loss: 0.0537 - accuracy: 0.9849 - val_loss: 0.0456 - val_accuracy: 0.9878

In [13]:
model.save("mask_detector.model", save_format="h5")

In [11]:
# I'm continuing the code after quite some time, so I'm going to load the model again

t_model = load_model("mask_detector.model")

In [15]:
predIdx = t_model.predict(x_test, BS)

predIdx = np.argmax(predIdx, axis=1)

26/26 [==============================] - 49s 2s/step


In [16]:
print(classification_report(y_test.argmax(axis=1), predIdx, target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       433
without_mask       0.99      0.99      0.99       386

    accuracy                           0.99       819
   macro avg       0.99      0.99      0.99       819
weighted avg       0.99      0.99      0.99       819

